Hi everyone! This is my first kaggle data analysis project. Here you are going to find some interesting details about Zomato delivery in Bangalore. So if you wanna know the biggest restaurant chains in Bangalore, you should scroll ahead :) Wanna know the most voted restaurants? Definitely Scroll ahead :) Wanna know Bangaloreans most relished dish? What are you waiting for???!! Scroll ahead already! :P 

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import missingno as msno
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
import math
from statistics import mode
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

First we load the data file.

In [ ]:

df=pd.read_csv('/kaggle/input/zomato-bangalore-restaurants/zomato.csv')

# GETTING FAMILIAR WITH THE DATASET
Now let's get familiar with the dataset. Calling first 5 rows...


In [ ]:
df.head()

Ok so we have basic details like name, location, cuisine info about restaurants. Now lets see what the datatypes of each columns are, how many null values are there in each, any discrepancies etc.


In [ ]:
df.info()

No of unique values in each column:

In [ ]:
df.nunique()

I am also curious about the type of entries in few columns. Some of them look like categorical values, so I'm going to check the values in them.

In [ ]:
df['listed_in(type)'].unique()

In [ ]:
df['dish_liked'].head()[0]

In [ ]:
df['reviews_list'][0]

Phew..that was lonnng

# DATA CLEANING
Here we'll clean the data before getting into analysis.
Okay, first thing,I see the menu_item column with empty list-like entries. So let's see what's happening there.

In [ ]:
df.menu_item.head()

Hmmm...that doesnt say anything about menus in restaurants.

Ok, I am reserving it to delete in the future. Meanwhile let's see how many null values are there in the dataset

In [ ]:
df.isnull().sum()

Let's visualise this with the help of missingno module. The white blanks show missing entries in the columns.

In [ ]:
msno.matrix(df)

Looks like there are quite a few missing values in the dish_liked column. Lets see how much

In [ ]:
(df['dish_liked'].isnull().sum())/(df.shape[0])

Wow,..that's more than 50%. So we must be cautious when dealing with entries from this column. 

Now I will delete the columns which are irrelevant to the analysis

In [ ]:
df.drop(['url','phone','menu_item'],axis=1,inplace=True)

In [ ]:
df.columns


Also lemme change the column titles to something easier and understandable for me.

In [ ]:

df.rename(columns={'listed_in(type)':'rest_category', 'listed_in(city)':'city','rate':'rating','approx_cost(for two people)':'cost_for_two'},inplace= True)

In [ ]:
df.head()

Removing any duplicate entries


In [ ]:

df.drop_duplicates(keep='last',inplace=True)



In [ ]:
df.duplicated().sum()

Now I'm going to change the rating column from string to float. That will suit us better when we need to do analysis to find relationships between restaurant rating and other factors


In [ ]:
df.rating.unique()
df.rating.replace('-',np.nan, inplace=True)
df.rating.replace('NEW',np.nan,inplace=True)
string_float = lambda x: x.replace('/5', '') if type(x) == np.str else x
df.rating = df.rating.apply(string_float).str.strip().astype('float')
df.rating.head()
       

In [ ]:
df.rating
type(df.rating[0])

Moving to the cost for two column. Same process. Also this time I see some column values less than 10. I dont believe there's a single restaurant in Bangalore where cost for 2 is 1.6 or 2.5 rupees. I am assuming that the data was entered 1.6 for 1600 and similarly. So I'll change <10 values to multiplied by 1000 values for accurate depiction

In [ ]:
df['cost_for_two'] = df['cost_for_two'].astype(str)
df['cost_for_two']=df['cost_for_two'].apply(lambda x: x.replace(',','.'))
df['cost_for_two']=df['cost_for_two'].astype(float)

In [ ]:
for i in df['cost_for_two'].unique():
    if i<10.0:
        df['cost_for_two'].replace({i:i*1000}, inplace=True)
      
        

In [ ]:
df['cost_for_two'].unique()

# ANALYSIS

I am soo curious to know which dish is most liked by bangaloreans,...so I can try it next time I visit bangalore!
Doing some manipulations...

In [ ]:
list=[]
for i in df['dish_liked']:
    if type(i)!=float:
        i.split(',')
        list.append(i)
# print(list)


In [ ]:
print(mode(list))

What!!!...what!..did I just do all that to figure out people eat biriyani the most ? Stupid me. Who needs data to know Indians love biriyani. Lets see the 10 most liked dishes.

In [ ]:
# 10 most liked dishes in bangalore
from collections import Counter
words_to_count = (word for word in list if word[:1].isupper())
c = Counter(words_to_count)
print (c.most_common(10))

...hmm,...whats friendly staff and rooftop ambience doing in this list?..looks like the data is corrupted. Moving on,..Here's a general statistical description of data

In [ ]:
df.describe()

Let's find which is the most voted restaurant

In [ ]:
#most voted
df[df['votes']==df['votes'].max()]

Yaay,..its **byg brewski**

Lets see the most expensive one in town

In [ ]:
#Most expensive
df[df['cost_for_two']==df['cost_for_two'].max()]

ahh..**Leela**, no surprise :D

Here's the most affordable one (let me note it down)

In [ ]:
#most affordable restaurant
df[df['cost_for_two']==df['cost_for_two'].min()]

Yaay,At 40 rs for 2, **srinidhi sagar** takes the cake.

Lets see the most popular cuisine categories offered by the restaurants.

In [ ]:
cuisine_list=[]
for i in df['cuisines']:
    if type(i)!=float:
        i.split(',')
        cuisine_list.append(i)
# print(cuisine_list)
new_list=[]
for i in cuisine_list:
    new_list.append(i.split(','))
new_new_list = []
for i in new_list:
    for j in i:
        new_new_list.append(j.strip())
# print(new_new_list)

from collections import Counter
words_to_count = (word for word in new_new_list if word[:1].isupper())
c = Counter(words_to_count)
print (c.most_common(10))

Hmm... despite being in the south Bangalore definitely loves **northy-food **:D

# VISUALISATIONS


Distribution plot of cost_for_two

In [ ]:
sns.distplot(df.cost_for_two,kde=False,bins=50)

Number of restaurants offering/not offering online order

In [ ]:
plt.figure()
sns.countplot(df.online_order)

Number of restaurants that offer/dont offer booking services

In [ ]:
sns.countplot(df.book_table)

Rating distribution

P.S: I noticed in priliminary analysis there were some null values in this column. So i checked the number of nulls before showing the distribution. The lesser the nulls the more accurate our representation is :)

In [ ]:
df.rating.isnull().sum()/df.shape[0]

I think, not bad. 

In [ ]:
fig,axes = plt.subplots(nrows=1, ncols=2, figsize=(12,4))
sns.distplot( df.rating, ax=axes[0])
sns.boxplot( df.rating, ax=axes[1])

That looks like 3.75-ish


Now let's see the category wise presence of restaurants on Zomato app.

In [ ]:
fig=plt.figure(figsize=(10,3))
sns.countplot(df.rest_category)
plt.tight_layout()

Location wise density of restaurants in Bangalore - top 10 locations with restaurants

In [ ]:
c= df.city.value_counts()[:10]
c.plot.bar()

Top 10 food types available in the app

In [ ]:
s= df.rest_type.value_counts()[:10]
s.plot.bar()

Relationship between table-booking service and rating. Looks like there's a positive correlation between them. Not so surprising. 

In [ ]:
bt_plot=pd.crosstab(df['rating'], df['book_table'])
bt_plot.plot(kind='bar',stacked=True);

Lets see the 10 most popular food outlet in Bangalore

In [ ]:
popular=df['name'].value_counts()[:10]
# popular.plot.bar()
sns.barplot(x=popular,y=popular.index,palette='Set1')

Haha,..CCD all the way. 

Also, one more curious thing I observed. Lemme know if you see it too ;)

In [ ]:
# most popular outlets
popular=df['name'].value_counts()[:20]
print(popular)

It looks like KFC , Domino's and Subway are in some kind of partnership to have no more outlets than each other in the city. Who knew of such sweet deals :D

Alright, that's it for now :) Let me know what you think. Also please forgive me for some of the laughable code inside. I know refactoring is a real thing, but I was too excited to share my first work :P 

Danke!

-Meera.